| python 에서는 다중상속이라는 편리한 기능이 있지만 그 때문에 골치아픈 경우가 발생합니다. 그를 피할 수 있는 방법이 MIX-IN 입니다.😊


### class 다중상속 시 문제
![](https://velog.velcdn.com/images/crosstar1228/post/02b4b24f-3c46-49d1-8bdf-6b4606fef887/image.png)

- A,B,C 는 수정 불가능한 패키지
- D,E 는 각각 B,C 로부터 상속받음

-> 이 경우, code duplicate가 일어남

## 해결 방안 : mixin
![](https://velog.velcdn.com/images/crosstar1228/post/f54dc318-5353-4aef-85c2-ee6d0b964781/image.png)

- 공통된 함수를 M이라는 mixin class 안에 정의함
- **자식 클래스가 사용할 METHOD 몇개만** 정의하는 클래스(attribute X)
- attribute 없으므로 __init__ method(constructor)도 필요없음

- 믹스인 클래스를 작성해두면 여러 클래스에 적용 가능함!([Generic](https://www.quora.com/What-are-generics-in-Python))



## `ToDictMixin` 클래스 정의

In [7]:
class ToDictMixin:
  def to_dict(self):
    return self._traverse_dict(self.__dict__)
    
  def _traverse_dict(self, instance_dict):
    output = {}
    for key, value in instance_dict.items():
      output[key] = self._traverse(key, value)
    return output

#  [Bool] = isinstance(확인하고자 하는 데이터 값, 확인하고자 하는 데이터 타입)

  def _traverse(self, key, value):
    if isinstance(value, ToDictMixin):
      return value.to_dict()
    if isinstance(value, dict):
      return self._traverse_dict(value)
    if isinstance(value, list):
      return [self._traverse(key, i) for i in value]
    elif hasattr(value, '__dict__'):
      return self._traverse_dict(value.__dict__)
    else:
      return value

### 이를 이용하여 이진트리-> 딕셔너리 변환!
- ToDictMixin 상속하여 `to_dict()` 

In [8]:
class BinaryTree(ToDictMixin): # mixin class 상속
  def __init__(self, value, left=None, right=None):
    self.value = value
    self.left = left
    self.right = right

tree = BinaryTree(10,
                  left = BinaryTree(7, right=BinaryTree(9)),
                  right = BinaryTree(13, left=BinaryTree(11))
                  )
print(tree.to_dict())

{'value': 10, 'left': {'value': 7, 'left': None, 'right': {'value': 9, 'left': None, 'right': None}}, 'right': {'value': 13, 'left': {'value': 11, 'left': None, 'right': None}, 'right': None}}


### 믹스인 장점 1 : override로 메소드 변경 가능
> 다음은 parent attribute가 추가된 클래스 `BinaryTreeWithParent`

In [16]:
class BinaryTreeWithParent(BinaryTree):
  def __init__(self, value, left=None, right=None, parent=None):
    super().__init__(value, left=left, right=right) #BinaryTree로부터 값 상속
    self.parent = parent

  ## overrriding 으로 순환 참조(무한루프)를 막음 -> to_dict() 사용가능
  def _traverse(self,key,value):
    if isinstance(value, BinaryTreeWithParent) and key=='parent':
      return value.value
    else:
      return super()._traverse(key,value)
    
  


In [17]:
root = BinaryTreeWithParent(10)
root.left = BinaryTreeWithParent(7, parent=root)
root.left.right = BinaryTreeWithParent(9, parent=root.left)
print(root.to_dict())

{'value': 10, 'left': {'value': 7, 'left': None, 'right': {'value': 9, 'left': None, 'right': None, 'parent': 7}, 'parent': 10}, 'right': None, 'parent': None}


이에 따라 `BinaryTreeWithParent`를 애트리뷰트로 저장하는 모든 클래스도 자동으로 `ToDictMixin`을 문제없이 사용하게 됨:

In [18]:
class NamedSubTree(ToDictMixin):
  def __init__(self, name, tree_with_parent):
    self.name = name
    self.tree_with_parent = tree_with_parent

my_tree = NamedSubTree('foobar', root.left.right)
print(my_tree.to_dict())

{'name': 'foobar', 'tree_with_parent': {'value': 9, 'left': None, 'right': None, 'parent': 7}}


## 믹스인 합성도 가능!

## Reference
- https://youtu.be/zNPaqZUTzs0
- http://pythonstudy.xyz/python/article/510-%EC%A7%81%EB%A0%AC%ED%99%94%EC%99%80-%EC%97%AD%EC%A7%81%EB%A0%AC%ED%99%94